In [1]:
!pip install translators
%pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import pandas as pd
import contractions
from datetime import datetime
import translators.server as tss
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Using region South Carolina server backend.


In [3]:
data = pd.read_csv('airpod_reviews.csv')

In [4]:
data.head()

,username_name,username_rating,username_inital_statement,username_country,username_verified,username_review,username_usefulness
0,Asia underwood,4.0 out of 5 stars,It has good noise cancellation the only proble...,Reviewed in the United States 🇺🇸 on February 1...,Verified Purchase,"so when I bought these , I knew what i wanted ...",7 people found this helpful
1,Katie,4.0 out of 5 stars,Love these Headphones,Reviewed in the United States 🇺🇸 on February 2...,Verified Purchase,I bought these headphones last year and they h...,6 people found this helpful
2,Sean Connolly,4.0 out of 5 stars,Solid,Reviewed in the United States 🇺🇸 on December 1...,Verified Purchase,"The noise cancellation feature is very cool, a...",6 people found this helpful
3,Michelle,5.0 out of 5 stars,BUY IT RN,Reviewed in the United States 🇺🇸 on January 17...,Verified Purchase,i was a little bit skeptical at first because ...,7 people found this helpful
4,Jayden,5.0 out of 5 stars,Greta product slightly dirty,Reviewed in the United States 🇺🇸 on February 1...,Verified Purchase,Everything works! The only down sid Eid there ...,NaN


In [5]:
data.count()

username_name                1996
username_rating              1996
username_inital_statement    1996
username_country             1996
username_verified            1941
username_review              1979
username_usefulness           424
dtype: int64

In [8]:
unique_names = data['username_name'].nunique()
unique_ratings = data['username_rating'].nunique()
unique_initial_statements = data['username_inital_statement'].nunique()
unique_reviews = data['username_review'].nunique()
unique_country = data['username_country'].nunique()
unique_verified = data['username_verified'].nunique()
unique_usefulness = data['username_usefulness'].nunique()

print('Unique names: ', unique_names)
print('Unique ratings: ', unique_ratings)
print('Unique initial statements: ', unique_initial_statements)
print('Unique reviews: ', unique_reviews)
print('Unique country: ', unique_country)
print('Unique verified: ', unique_verified)
print('Unique usefulness: ', unique_usefulness)

Unique names:  1813
Unique ratings:  5
Unique initial statements:  1683
Unique reviews:  1928
Unique country:  976
Unique verified:  2
Unique usefulness:  35


In [24]:
data = data.rename(columns={
    'username_name':'username',
    'username_rating':'rating',
    'username_inital_statement':'initial_statement',
    'username_verified':'verified',
    'username_review':'review',
    'username_usefulness':'usefulness',
    'username_country': 'country'
})
data.head()

,username,rating,initial_statement,country,verified,review,usefulness
0,Asia underwood,4.0 out of 5 stars,It has good noise cancellation the only proble...,Reviewed in the United States 🇺🇸 on February 1...,Verified Purchase,"so when I bought these , I knew what i wanted ...",7 people found this helpful
1,Katie,4.0 out of 5 stars,Love these Headphones,Reviewed in the United States 🇺🇸 on February 2...,Verified Purchase,I bought these headphones last year and they h...,6 people found this helpful
2,Sean Connolly,4.0 out of 5 stars,Solid,Reviewed in the United States 🇺🇸 on December 1...,Verified Purchase,"The noise cancellation feature is very cool, a...",6 people found this helpful
3,Michelle,5.0 out of 5 stars,BUY IT RN,Reviewed in the United States 🇺🇸 on January 17...,Verified Purchase,i was a little bit skeptical at first because ...,7 people found this helpful
4,Jayden,5.0 out of 5 stars,Greta product slightly dirty,Reviewed in the United States 🇺🇸 on February 1...,Verified Purchase,Everything works! The only down sid Eid there ...,NaN


In [25]:
data = data.drop(['username', 'verified', 'usefulness'], axis = 1)
data.head()

,rating,initial_statement,country,review
0,4.0 out of 5 stars,It has good noise cancellation the only proble...,Reviewed in the United States 🇺🇸 on February 1...,"so when I bought these , I knew what i wanted ..."
1,4.0 out of 5 stars,Love these Headphones,Reviewed in the United States 🇺🇸 on February 2...,I bought these headphones last year and they h...
2,4.0 out of 5 stars,Solid,Reviewed in the United States 🇺🇸 on December 1...,"The noise cancellation feature is very cool, a..."
3,5.0 out of 5 stars,BUY IT RN,Reviewed in the United States 🇺🇸 on January 17...,i was a little bit skeptical at first because ...
4,5.0 out of 5 stars,Greta product slightly dirty,Reviewed in the United States 🇺🇸 on February 1...,Everything works! The only down sid Eid there ...


In [26]:
data['full_review'] = data['initial_statement'] + ". " + data['review']
data = data.drop(['initial_statement', 'review'], axis = 1)
data.head()

,rating,country,full_review
0,4.0 out of 5 stars,Reviewed in the United States 🇺🇸 on February 1...,It has good noise cancellation the only proble...
1,4.0 out of 5 stars,Reviewed in the United States 🇺🇸 on February 2...,Love these Headphones. I bought these headphon...
2,4.0 out of 5 stars,Reviewed in the United States 🇺🇸 on December 1...,Solid. The noise cancellation feature is very ...
3,5.0 out of 5 stars,Reviewed in the United States 🇺🇸 on January 17...,BUY IT RN. i was a little bit skeptical at fir...
4,5.0 out of 5 stars,Reviewed in the United States 🇺🇸 on February 1...,Greta product slightly dirty. Everything works...


In [27]:
data['date'] = data['country'].str.split(" ").str[-3:].str.join(" ")
data["date"] = pd.to_datetime(data["date"])
data.head()

,rating,country,full_review,date
0,4.0 out of 5 stars,Reviewed in the United States 🇺🇸 on February 1...,It has good noise cancellation the only proble...,2023-02-01
1,4.0 out of 5 stars,Reviewed in the United States 🇺🇸 on February 2...,Love these Headphones. I bought these headphon...,2023-02-02
2,4.0 out of 5 stars,Reviewed in the United States 🇺🇸 on December 1...,Solid. The noise cancellation feature is very ...,2022-12-12
3,5.0 out of 5 stars,Reviewed in the United States 🇺🇸 on January 17...,BUY IT RN. i was a little bit skeptical at fir...,2023-01-17
4,5.0 out of 5 stars,Reviewed in the United States 🇺🇸 on February 1...,Greta product slightly dirty. Everything works...,2023-02-15


In [28]:
data['country'] = data['country'].str.split(" ").str[3:5].str.join(" ")
data.head()

,rating,country,full_review,date
0,4.0 out of 5 stars,United States,It has good noise cancellation the only proble...,2023-02-01
1,4.0 out of 5 stars,United States,Love these Headphones. I bought these headphon...,2023-02-02
2,4.0 out of 5 stars,United States,Solid. The noise cancellation feature is very ...,2022-12-12
3,5.0 out of 5 stars,United States,BUY IT RN. i was a little bit skeptical at fir...,2023-01-17
4,5.0 out of 5 stars,United States,Greta product slightly dirty. Everything works...,2023-02-15


In [29]:
data['country'].unique()

array(['United States', '🇲🇽 on', '🇨🇦 on'], dtype=object)

In [30]:
data['country'] = data['country'].replace({'🇲🇽 on':'Mexico', '🇨🇦 on':'Canada'})
data['country'].unique()

array(['United States', 'Mexico', 'Canada'], dtype=object)

In [31]:
data = data.dropna(subset=['full_review'])
data['full_review'].isnull().sum()

0

In [32]:
date_range = data['date'].min().strftime('%Y-%m-%d') + ' to ' + data['date'].max().strftime('%Y-%m-%d')
date_range

'2019-12-17 to 2023-02-21'

In [33]:
data['rating'] = data['rating'].str[0]
data['rating'].astype(int)
data.head()

,rating,country,full_review,date
0,4,United States,It has good noise cancellation the only proble...,2023-02-01
1,4,United States,Love these Headphones. I bought these headphon...,2023-02-02
2,4,United States,Solid. The noise cancellation feature is very ...,2022-12-12
3,5,United States,BUY IT RN. i was a little bit skeptical at fir...,2023-01-17
4,5,United States,Greta product slightly dirty. Everything works...,2023-02-15


In [34]:
def translate(text):
    return tss.google(text, from_language="es", to_language="en")

In [35]:
data['full_review'] = data['full_review'].apply(translate)
data[data['country'] == 'Mexico']

HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://translate.google.com/_/TranslateWebserverUi/data/batchexecute&q=EhAmBa2AABHR_TcA4AzcvylMGKC94aEGIjBt-iKc39MSoXwjGw_4qSI7mRjBYT5o4_v1pozV4gMCdAri8S-9k-BBreeMbjLKd4YyAXI

In [ ]:
data['full_review'] = data['full_review'].str.lower()

In [ ]:
def contraction_extender(text):
    return contractions.fix(text)

In [ ]:
data['full_review'] = data['full_review'].apply(contraction_extender)
data.head()

,rating,country,full_review,date
0,4,United States,it has good noise cancellation the only proble...,2023-02-01
1,4,United States,love the headphones. i boought these headphone...,2023-02-02
2,4,United States,solid. the noise cancellation feature is vry c...,2022-12-12
3,5,United States,buy it right now. i was a little skeptical bit...,2023-01-17
4,5,United States,greta product slightly dirty. everything works...,2023-02-15


In [ ]:
product_keywords = ['quality', 'condition', 'product', 'cost', 'build', 'design', 'durability', 'material', 'materials', 'size', 'weight', 'performs', 'performance', 'work', 'works', 'style', 'safe', 'safety', 'look', 'looks', 'overall', 'worked', 'recommend', 'recommended', 'use', 'expect', 'expected', 'inexpensive', 'issue', 'easy''love', 'cute', 'great', 'comfortable', 'good', 'nice', 'excellent', 'awesome', 'amazing', 'perfect', 'best', 'beautiful', 'beautifully', 'well', 'well made', 'well', 'stylish', 'durable', 'light weight', 'lightweight', 'compact', 'coomfortable', 'comfy', 'impressed', 'impress', 'lovely', 'fit', 'fits', 'beautiful', 'affordable', 'comfortably', 'stunning', 'classy', 'excellent', 'poor', 'uncomfortable', 'poorly', 'bad', 'terrible', 'worst', 'disappointed', 'disappointing', 'disappoint', 'disappointment', 'disappointments', 'expensive', 'pricely', 'downside', ]
seller_keywords = ['customer service', 'reply', 'confirmation', 'response', 'responded', 'refund', 'refunds', 'experience', 'as advertised', 'positive', 'responsive', 'professional', 'friendly', 'transparent', 'transparency', 'helpful', 'respectful', 'polite', 'courteous', 'nice', 'positive experience', 'kind''rude', 'unresponsive', 'unprofessional', 'unfriendly', 'unhelpful', 'negative', 'disrespectful', 'impolite', 'not courteous', 'not respectful', 'not polite', 'not helpful', 'not friendly', 'not responsive', 'not professional', 'not transparent', 'poor', 'service']
shipping_keywords = ['came', 'arrived', 'delivered', 'shipped', 'shipping', 'delivery', 'receive', 'received', 'package', 'packaging', 'days', 'track', 'tracking', 'tracked', 'tracker', 'day', 'overdue', 'returning', 'lost', 'late', 'damaged', 'delayed', 'delay', 'delays', 'slow', 'slowly', 'slow delivery', 'slowly', 'slowly delivered', 'slow delivery', 'slowly delivered', 'slowly delivery', 'mishandled', 'fast', 'quick', 'quickly', 'timely', 'trackable', 'well packaged', 'well-packaged', 'on-time', 'on time', 'speedy', 'well-packaged', 'well packaged', 'swift', 'missing']

In [ ]:
def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(text)
    return sentiment

In [ ]:
def predict(text):
    sentiment = analyze_sentiment(text)
    delimiters = ".!?"
    review = re.split('['+delimiters+']', text)
    
    total_seller_rating = 0
    total_product_rating = 0
    total_shipping_rating = 0
    seller_counter = 0
    product_counter = 0
    shipping_counter = 0

    for sentence in review:
        for keyword in seller_keywords:
            if keyword in sentence:
                total_seller_rating += sentiment['compound']
                seller_counter += 1
            else:
                seller_counter += 0
        for keyword in product_keywords:
            if keyword in sentence:
                total_product_rating += sentiment['compound']
                product_counter += 1
            else:
                product_counter += 0
        for keyword in shipping_keywords:
            if keyword in sentence:
                total_shipping_rating += sentiment['compound']
                shipping_counter += 1
            else:
                shipping_counter += 0

    if seller_counter == 0:
        seller_rating = 0
    else:
        seller_rating = total_seller_rating / seller_counter
        if -1 <= seller_rating <= -0.525:
            seller_rating = 1
        elif -0.525 < seller_rating <= -0.05:
            seller_rating = 2
        elif -0.05 < seller_rating <= 0.05:
            seller_rating = 3
        elif 0.05 < seller_rating <= .525:
            seller_rating = 4
        elif 0.525 < seller_rating <= 1:
            seller_rating = 5

    if product_counter == 0:
        product_rating = 0
    else:
        product_rating = total_product_rating / product_counter
        if -1 <= product_rating <= -0.525:
            product_rating = 1
        elif -0.525 < product_rating <= -0.05:
            product_rating = 2
        elif -0.05 < product_rating <= 0.05:
            product_rating = 3
        elif 0.05 < product_rating <= .525:
            product_rating = 4
        elif 0.525 < product_rating <= 1:
            product_rating = 5

    if shipping_counter == 0:
        shipping_rating = 0
    else:
        shipping_rating = total_shipping_rating / shipping_counter
        if -1 <= shipping_rating <= -0.525:
            shipping_rating = 1
        elif -0.525 < shipping_rating <= -0.05:
            shipping_rating = 2
        elif -0.05 < shipping_rating <= 0.05:
            shipping_rating = 3
        elif 0.05 < shipping_rating <= .525:
            shipping_rating = 4
        elif 0.525 < shipping_rating <= 1:
            shipping_rating = 5

    ratings = {
        'seller_rating': seller_rating,
        'product_rating': product_rating,
        'shipping_rating': shipping_rating,
    }

    return ratings

In [ ]:
ratings_df = data['full_review'].apply(predict).apply(pd.Series)
data = pd.concat([data, ratings_df], axis=1)
data.head()

,rating,country,full_review,date,seller_rating,product_rating,shipping_rating
0,4,United States,it has good noise cancellation the only proble...,2023-02-01,0,2,2
1,4,United States,love the headphones. i boought these headphone...,2023-02-02,0,5,0
2,4,United States,solid. the noise cancellation feature is vry c...,2022-12-12,0,5,0
3,5,United States,buy it right now. i was a little skeptical bit...,2023-01-17,0,5,5
4,5,United States,greta product slightly dirty. everything works...,2023-02-15,0,2,2


In [ ]:
data.dtypes

rating                     object
country                    object
full_review                object
date               datetime64[ns]
seller_rating               int64
product_rating              int64
shipping_rating             int64
dtype: object

In [ ]:
data.head()

NameError: name 'data' is not defined